In [1]:
import os, sys
import django
import pytz
from dateutil import parser
TIME_ZONE = 'Africa/Gaborone'
tz = pytz.timezone(TIME_ZONE)

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "tshilo_dikotla.settings"

# Connect to Django ORM
django.setup()
import uuid
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

try:
    td = Site.objects.get(id=40)
except Site.DoesNotExist:
    td = Site.objects.create(id=40, domain='td-live.bhp.org.bw', name='td-live.bhp.org.bw')

In [2]:
# Subject Screening
from td_maternal.models import SubjectScreening
from td_maternal.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/maternal_eligibility.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)

site = Site.objects.get(id=40)
count = 0
already_created = 0
for sc in sc_data:
    del sc['_state']
    del sc['registered_subject_id']
    sc.update(
        report_datetime=parser.parse(sc.get('report_datetime')).replace(tzinfo=tz),
        created=parser.parse(sc.get('created')).replace(tzinfo=tz),
        site=site,
        id=uuid.UUID(sc.get('id'))
    )
    del sc['eligibility_id']
    del sc['export_change_type']
    del sc['export_uuid']
    del sc['exported']
    del sc['exported_datetime']
    if sc['modified']:
        sc.update(modified=parser.parse(sc.get('modified')).replace(tzinfo=tz))
    else:
        sc.update(modified=None)
    try:
        SubjectScreening.objects.get(id=sc.get('id'))
    except SubjectScreening.DoesNotExist:
        sc.update(screening_identifier=ScreeningIdentifier().identifier)
        subject_Screening = SubjectScreening(**sc)
        subject_Screening.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created subject screening {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sc_data))
print("Existing: ", SubjectScreening.objects.all().count())

Total created subject screening 0
Already created 555
Expected:  555
Existing:  555


In [3]:
# TD Consent Version
import uuid
from td_maternal.models import TdConsentVersion
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/td_consent_version.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

td_consent_version_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    td_consent_version_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in td_consent_version_data:
    del dd['_state']
    created = parser.parse(dd.get('created'))
    created = created.replace(tzinfo=tz)
    dd.update(
        created=created,
        report_datetime=parser.parse(dd.get('report_datetime')).replace(tzinfo=tz),
        id=uuid.UUID(dd.get('id'))
    )
    maternal_eligibility_id = uuid.UUID(dd.get('maternal_eligibility_id'))
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = modified.replace(tzinfo=tz)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    try:
        subject_screening = SubjectScreening.objects.get(id=maternal_eligibility_id)
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Subject Screening can not be missing ', maternal_eligibility_id)
    else:
        del dd['maternal_eligibility_id']
        screening_identifier = subject_screening.screening_identifier
        try:
            TdConsentVersion.objects.get(screening_identifier=screening_identifier)
        except TdConsentVersion.DoesNotExist:
            dd.update(screening_identifier=screening_identifier)
            td_consent_version = TdConsentVersion(**dd)
            td_consent_version.save_base(raw=True)
            count += 1
        else:
            already_created += 1
print(f'Total created td_consent_version {count}')
print(f'Already created td_consent_version {already_created}')
print("Expected: ", len(td_consent_version_data))
print("Existing: ", TdConsentVersion.objects.all().count())

Total created td_consent_version 0
Already created td_consent_version 456
Expected:  456
Existing:  456


In [4]:
# Subject Consent
import uuid
from td_maternal.models import SubjectConsent
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/maternal_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

s_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    s_consent_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in s_consent_data:
    del dd['_state']
    del dd['export_change_type']
    del dd['export_uuid']
    del dd['exported']
    del dd['exported_datetime']
    del dd['study_site']
    dd.update(site=td)
    created = parser.parse(dd.get('created'))
    created = created.replace(tzinfo=tz)
    consent_datetime = parser.parse(dd.get('consent_datetime'))
    consent_datetime = consent_datetime.replace(tzinfo=tz)
    dob = parser.parse(dd.get('dob')).replace(tzinfo=tz)
    dob = dob.date()
    dd.update(
        created=created,
        consent_datetime=consent_datetime,
        report_datetime=parser.parse(dd.get('consent_datetime')).replace(tzinfo=tz),
        dob=dob,
        id=uuid.UUID(dd.get('id'))
    )
    if dd['subject_identifier_as_pk']:
        dd.update(
            subject_identifier_as_pk=uuid.UUID(dd.get('subject_identifier_as_pk')))
    else:
        dd.update(
            subject_identifier_as_pk=dd.get('id'))
    if dd['is_verified_datetime']:
        is_verified_datetime = parser.parse(dd.get('is_verified_datetime'))
        is_verified_datetime = is_verified_datetime.replace(tzinfo=tz)
        dd.update(
        is_verified_datetime=is_verified_datetime)
    else:
        dd.update(is_verified_datetime=None)
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = modified.replace(tzinfo=tz)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    if dd['is_verified'] == 'TRUE':
        dd.update(is_verified=True)
    elif dd['is_verified'] == 'FALSE':
        dd.update(is_verified=False)
    if dd['guardian_name']:
        guardian_name = dd['guardian_name']
        name1, name2 = guardian_name.split(',')
        guardian_name = name1 + ', ' + name2
        dd.update(guardian_name=guardian_name)
    try:
        subject_screening = SubjectScreening.objects.get(id=dd['maternal_eligibility_id'])
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Eligibility can not be missing for this subject', dd['subject_identifier'])
    else:
        dd.update(screening_identifier=subject_screening.screening_identifier)
        del dd['maternal_eligibility_id']
    try:
        SubjectConsent.objects.get(id=dd.get('id'))
    except SubjectConsent.DoesNotExist:
        subject_consent = SubjectConsent(**dd)
        subject_consent.save_base(raw=True)
        count += 1
        subject_screening.subject_identifier = dd['subject_identifier']
        subject_screening.save_base(raw=True)
    else:
        already_created += 1
print(f'Total created consents {count}')
print(f'Already created {already_created}')
print("Expected: ", len(s_consent_data))
print("Existing: ", SubjectConsent.objects.all().count())

Total created consents 0
Already created 905
Expected:  905
Existing:  905


In [5]:
# Registered Subject
from edc_registration.models import RegisteredSubject

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/registered_subject.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

rs_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        raise ValidationError('Line is not the right size')
    rs_data.append(data)

td = Site.objects.get(id=40)
already_created = 0
count = 0
for rs in rs_data:
    del rs['may_store_samples']
    del rs['hiv_status']
    rs.update(site=td)
    del rs['survival_status']
    del rs['_state']
    del rs['max_subjects']
    del rs['study_site']
    del rs['subject_types']
    rs.update(
        created=parser.parse(rs.get('created')).replace(tzinfo=tz),
        id=uuid.UUID(rs.get('id'))
    )
    if rs['dob']:
        rs.update(dob=parser.parse(rs.get('dob')).replace(tzinfo=tz).date())
    else:
        rs.update(dob=None)
    if rs['modified']:
        rs.update(modified=parser.parse(rs.get('modified')).replace(tzinfo=tz))
    else:
        rs.update(modified=None)
    if rs['screening_datetime']:
        rs.update(
            screening_datetime=parser.parse(rs.get('screening_datetime')).replace(tzinfo=tz))
    else:
        rs.update(screening_datetime=None)
    if rs['registration_datetime']:
        rs.update(
            registration_datetime=parser.parse(rs.get('registration_datetime')).replace(tzinfo=tz))
    else:
        rs.update(registration_datetime=None)
    if rs['randomization_datetime']:
        rs.update(
            randomization_datetime=parser.parse(rs.get('randomization_datetime')).replace(tzinfo=tz))
    else:
        rs.update(randomization_datetime=None)
    if rs['screening_age_in_years']:
        rs.update(screening_age_in_years=int(float(rs['screening_age_in_years'])))
    else:
        rs.update(screening_age_in_years=None)
    if not rs['subject_identifier'][-3:] == '-10' and not len(rs['subject_identifier']) == 36:
        try:
            subject_screening = SubjectScreening.objects.get(subject_identifier=rs['subject_identifier'])
        except SubjectScreening.DoesNotExist:
            raise ValidationError('Subject screening can not be missing', rs['subject_identifier'])
        else:
            rs.update(screening_identifier=subject_screening.screening_identifier)
    try:
        RegisteredSubject.objects.get(subject_identifier=rs.get('subject_identifier'))
    except RegisteredSubject.DoesNotExist:
        registered_subject = RegisteredSubject(**rs)
        registered_subject.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print('Total created registered subjects: ', count)
print('Already created: ', already_created)
print("Expected: ", len(rs_data))
print("Existing: ", RegisteredSubject.objects.all().count())

Total created registered subjects:  0
Already created:  1016
Expected:  1016
Existing:  1016


In [6]:
# TD Consent Version
import uuid
from td_maternal.models import AntenatalEnrollment
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/antenatal_enrollment.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

antenatal_enrollment_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    antenatal_enrollment_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
skip_list = ['085-40990560-4', '085-40990487-1', '085-40990402-0']
already_created = 0
for ae in antenatal_enrollment_data:
    del ae['_state']
    created = parser.parse(ae.get('created'))
    created = created.replace(tzinfo=tz)
    ae.update(
        created=created,
        report_datetime=parser.parse(ae.get('report_datetime')).replace(tzinfo=tz),
        id=uuid.UUID(ae.get('id'))
    )
    if ae['modified']:
        modified = parser.parse(ae.get('modified'))
        modified = modified.replace(tzinfo=tz)
        ae.update(modified=modified)
    else:
        ae.update(modified=None)
    del ae['consent_version']
    del ae['export_change_type']
    del ae['export_uuid']
    del ae['exported']
    del ae['exported_datetime']
    del ae['registered_subject_id']
    del ae['_registered_subject_cache']
    if ae['date_at_32wks']:
        ae.update(date_at_32wks=parser.parse(ae.get('date_at_32wks')).replace(tzinfo=tz).date())
    else:
        ae.update(date_at_32wks=None)
    if ae['edd_by_lmp']:
        ae.update(edd_by_lmp=parser.parse(ae.get('edd_by_lmp')).replace(tzinfo=tz).date())
    else:
        ae.update(edd_by_lmp=None)
    if ae['last_period_date']:
        ae.update(last_period_date=parser.parse(ae.get('last_period_date')).replace(tzinfo=tz).date())
    else:
        ae.update(last_period_date=None)
    if ae['rapid_test_date']:
        ae.update(rapid_test_date=parser.parse(ae.get('rapid_test_date')).replace(tzinfo=tz).date())
    else:
        ae.update(rapid_test_date=None)
    if ae['week32_test_date']:
        ae.update(week32_test_date=parser.parse(ae.get('week32_test_date')).replace(tzinfo=tz).date())
    else:
        ae.update(week32_test_date=None)
    if ae['ga_lmp_anc_wks']:
        ae.update(ga_lmp_anc_wks=int(float(ae['ga_lmp_anc_wks'])))
    else:
        ae.update(ga_lmp_anc_wks=None)
    if ae['ga_lmp_enrollment_wks']:
        ae.update(ga_lmp_enrollment_wks=int(float(ae['ga_lmp_enrollment_wks'])))
    else:
        ae.update(ga_lmp_enrollment_wks=None)
    try:
        AntenatalEnrollment.objects.get(subject_identifier=ae.get('subject_identifier'))
    except AntenatalEnrollment.DoesNotExist:
        if not ae.get('subject_identifier') in skip_list:
            antenatal_enrollment = AntenatalEnrollment(**ae)
            antenatal_enrollment.save_base(raw=True)
            count += 1
    else:
        already_created += 1
print(f'Total created antenatal_enrollment {count}')
print(f'Already created antenatal_enrollment {already_created}')
print("Expected: ", len(antenatal_enrollment_data))
print("Existing: ", AntenatalEnrollment.objects.all().count())

Total created antenatal_enrollment 0
Already created antenatal_enrollment 552
Expected:  555
Existing:  552


In [7]:
# Antenatal Visit Membership
import uuid
from td_maternal.models import AntenatalVisitMembership
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/antenatal_visit_membership.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

antenatal_visit_membership_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    antenatal_visit_membership_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for avm in antenatal_visit_membership_data:
    del avm['_state']
    created = parser.parse(avm.get('created'))
    created = created.replace(tzinfo=tz)
    avm.update(
        created=created,
        report_datetime=parser.parse(avm.get('report_datetime')).replace(tzinfo=tz),
        id=uuid.UUID(avm.get('id'))
    )
    if avm['modified']:
        modified = parser.parse(avm.get('modified'))
        modified = modified.replace(tzinfo=tz)
        avm.update(modified=modified)
    else:
        avm.update(modified=None)
    del avm['consent_version']
    del avm['export_change_type']
    del avm['export_uuid']
    del avm['exported']
    del avm['exported_datetime']
    del avm['registered_subject_id']
    del avm['_registered_subject_cache']
    try:
        AntenatalVisitMembership.objects.get(subject_identifier=avm.get('subject_identifier'))
    except AntenatalVisitMembership.DoesNotExist:
        antenatal_visit_membership = AntenatalVisitMembership(**avm)
        antenatal_visit_membership.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created antenatal_visit_membership {count}')
print(f'Already created antenatal_visit_membership {already_created}')
print("Expected: ", len(antenatal_visit_membership_data))
print("Existing: ", AntenatalVisitMembership.objects.all().count())

Total created antenatal_visit_membership 0
Already created antenatal_visit_membership 510
Expected:  510
Existing:  510


In [8]:
# Specimen Consent
import uuid
from td_maternal.models import SpecimenConsent
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/specimen_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

specimen_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    specimen_consent_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for sc in specimen_consent_data:
    del sc['_state']
    created = parser.parse(sc.get('created'))
    created = created.replace(tzinfo=tz)
    sc.update(
        created=created,
        id=uuid.UUID(sc.get('id'))
    )
    if sc['consent_datetime']:
        consent_datetime = parser.parse(sc.get('consent_datetime'))
        consent_datetime = consent_datetime.replace(tzinfo=tz)
        sc.update(consent_datetime=consent_datetime)
    else:
        sc.update(consent_datetime=None)
    if sc['modified']:
        modified = parser.parse(sc.get('modified'))
        modified = modified.replace(tzinfo=tz)
        sc.update(modified=modified)
    else:
        sc.update(modified=None)
    if sc['is_verified_datetime']:
        is_verified_datetime = parser.parse(sc.get('is_verified_datetime'))
        is_verified_datetime = is_verified_datetime.replace(tzinfo=tz)
        sc.update(
        is_verified_datetime=is_verified_datetime)
    else:
        sc.update(is_verified_datetime=None)
    if sc['is_verified'] == 'TRUE':
        sc.update(is_verified=True)
    elif sc['is_verified'] == 'FALSE':
        sc.update(is_verified=False)
    if sc['offered_copy']:
        sc.update(consent_copy=sc['offered_copy'])
    else:
        sc.update(consent_copy=None)
    del sc['offered_copy']
    if sc['purpose_explained']:
        sc.update(consent_reviewed=sc['purpose_explained'])
    else:
        sc.update(consent_reviewed=None)
    if sc['purpose_understood']:
        sc.update(assessment_score=sc['purpose_understood'])
    else:
        sc.update(assessment_score=None)
    del sc['purpose_understood']
    del sc['purpose_explained']
    del sc['version']
    del sc['export_change_type']
    del sc['export_uuid']
    del sc['exported']
    del sc['exported_datetime']
    del sc['registered_subject_id']
    del sc['_registered_subject_cache']
    try:
        SpecimenConsent.objects.get(subject_identifier=sc.get('subject_identifier'))
    except SpecimenConsent.DoesNotExist:
        specimen_consent = SpecimenConsent(**sc)
        specimen_consent.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created specimen_consent {count}')
print(f'Already created specimen_consent {already_created}')
print("Expected: ", len(specimen_consent_data))
print("Existing: ", SpecimenConsent.objects.all().count())

Total created specimen_consent 0
Already created specimen_consent 555
Expected:  555
Existing:  555


In [9]:
# Import Locators
from td_maternal.models import MaternalLocator, SubjectConsent
from edc_action_item.models import ActionItem

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/maternal_locator.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sl_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    sl_data.append(data)
    count += 1

td = Site.objects.get(id=40)
already_created = 0
count = 0
infant_locators = []
for sl in sl_data:
    del sl['_registered_subject_cache']
    del sl['registered_subject_id']
    del sl['_state']
    del sl['export_change_type']
    del sl['export_uuid']
    del sl['exported']
    del sl['exported_datetime']
    sl.update(site=td)
    sl.update(
        created=parser.parse(sl.get('created')).replace(tzinfo=tz),
        id=uuid.UUID(sl.get('id'))
    )
    if sl['may_sms_follow_up']:
        sl.update(may_sms=sl.get('may_sms_follow_up'))
        del sl['may_sms_follow_up']
    else:
        del sl['may_sms_follow_up']
    if sl['may_contact_someone']:
        sl.update(may_contact_indirectly=sl.get('may_contact_someone'))
        del sl['may_contact_someone']
    else:
        del sl['may_contact_someone']
    if sl['contact_name']:
        sl.update(indirect_contact_name=sl.get('contact_name'))
        del sl['contact_name']
    else:
        del sl['contact_name']
    if sl['contact_rel']:
        sl.update(indirect_contact_relation=sl.get('contact_rel'))
        del sl['contact_rel']
    else:
        del sl['contact_rel']
    if sl['contact_physical_address']:
        sl.update(indirect_contact_physical_address=sl.get('contact_physical_address'))
        del sl['contact_physical_address']
    else:
        del sl['contact_physical_address']
    if sl['contact_cell']:
        sl.update(indirect_contact_cell=sl.get('contact_cell'))
        del sl['contact_cell']
    else:
        del sl['contact_cell']
    if sl['contact_phone']:
        sl.update(indirect_contact_phone=sl.get('contact_phone'))
        del sl['contact_phone']
    else:
        del sl['contact_phone']
    if sl['subject_work_phone']:
        sl.update(subject_work_cell=sl.get('subject_work_phone'))
        del sl['subject_work_phone']
    else:
        del sl['subject_work_phone']
    if sl['care_clinic']:
        sl.update(health_care_infant=sl.get('care_clinic'))
        del sl['care_clinic']
    else:
        del sl['care_clinic']
    if sl['may_follow_up']:
        sl.update(may_call=sl.get('may_follow_up'))
        del sl['may_follow_up']
    else:
        del sl['may_follow_up']
    if sl['home_visit_permission']:
        sl.update(may_visit_home=sl.get('home_visit_permission'))
        del sl['home_visit_permission']
    else:
        del sl['home_visit_permission']
    if sl['modified']:
        sl.update(modified=parser.parse(sl.get('modified')).replace(tzinfo=tz))
    else:
        sl.update(modified=None)
    if sl['report_datetime']:
        sl.update(report_datetime=parser.parse(sl.get('report_datetime')).replace(tzinfo=tz))
    else:
        sl.update(report_datetime=None)
    if sl['date_signed']:
        sl.update(locator_date=parser.parse(sl.get('date_signed')).replace(tzinfo=tz))
        del sl['date_signed']
    else:
        sl.update(locator_date=None)
        del sl['date_signed']
    if sl.get('subject_identifier')[-3:] == '-10':
        infant_locators.append(sl.get('subject_identifier'))
    else:
        try:
            MaternalLocator.objects.get(subject_identifier=sl.get('subject_identifier'))
        except MaternalLocator.DoesNotExist:
            subject_locator = MaternalLocator(**sl)
            subject_locator.save()
            count += 1
        else:
            already_created += 1
print(f'Total created subject locator {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sl_data))
print("Existing: ", MaternalLocator.objects.all().count())

Total created subject locator 0
Already created 544
Expected:  550
Existing:  544


In [10]:
infant_locators

['085-40990064-5-10',
 '085-40990196-4-10',
 '085-40990209-3-10',
 '085-40990250-2-10',
 '085-40990223-3-10',
 '085-40990274-5-10']

In [11]:
# Import appointments
from edc_appointment.models import Appointment

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/appointments.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

schedule_n_visit_schedule = [
    ['1000M', 'V3', 'antenatal_schedule_3', 'anv_schedule_v3'],
    ['1000M', 'V1', 'antenatal_schedule_1', 'anv_schedule_v1'],
    ['1010M', 'V1', 'anv_membership_v1', 'anv_membership_v1'],
    ['1010M', 'V3', 'anv_membership_v3', 'anv_membership_v3'],
    ['1020M', 'V3', 'anv_membership_v3', 'anv_membership_v3'],
    ['1020M', 'V1', 'anv_membership_v1', 'anv_membership_v1'],
    ['2000M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2000M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2010M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2010M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2020M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2020M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2060M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2060M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2120M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2120M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2180M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2180M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2240M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2240M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2300M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2300M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2360M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2360M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3']]

ap_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    ap_data.append(data)
site = Site.objects.all()[0]
count = 0
already_created = 0
for ap in ap_data:
    if not sl.get('subject_identifier')[-3:] == '-10':
        ap.update(
            created=parser.parse(ap.get('created')).replace(tzinfo=tz),
            id=uuid.UUID(ap.get('id')),
            facility_name='5-day clinic',
        )
        instruction = ap.get('instruction')
        for code, instruction, schedule, visit_schedule in schedule_n_visit_schedule:
            if code == ap.get('visit_code') and instruction == ap.get('instruction'):
                ap.update(
                    schedule_name=schedule,
                    visit_schedule_name=visit_schedule
                )
        ap.update(timepoint=float(ap['timepoint']))
        if ap['modified']:
            ap.update(modified=parser.parse(ap.get('modified')).replace(tzinfo=tz))
        else:
            ap.update(modified=None)
        if ap['appt_status']:
            ap.update(appt_status=ap.get('appt_status'))
        else:
            ap.update(appt_status=None)
        if ap['appt_datetime']:
            ap.update(
            appt_datetime=parser.parse(ap.get('appt_datetime')).replace(tzinfo=tz))
        else:
            ap.update(appt_datetime=None)
        
        if ap['timepoint_datetime']:
            ap.update(
            timepoint_datetime=parser.parse(ap.get('timepoint_datetime')).replace(tzinfo=tz))
        else:
            ap.update(timepoint_datetime=None)
        
        if ap['appt_close_datetime']:
            ap.update(
            appt_close_datetime=parser.parse(ap.get('appt_close_datetime')).replace(tzinfo=tz))
        else:
            ap.update(appt_close_datetime=None)
        ap.update(site=Site.objects.get(id=40))
        if ap['is_confirmed'] == 'False':
            ap.update(is_confirmed=False)
        elif ap['is_confirmed'] == 'True':
            ap.update(is_confirmed=True)
        if ap['visit_instance']:
            ap.update(visit_code_sequence=int(ap['visit_instance']))
        else:
            ap.update(visit_code_sequence=0)
        del ap['contact_tel']
        del ap['contact_count']
        del ap['visit_instance']
        del ap['_registered_subject_cache']
        del ap['_state']
        del ap['dashboard_type']
        del ap['_time_point_status_cache']
        del ap['_visit_definition_cache']
        del ap['visit_definition_id']
        del ap['registered_subject_id']
        del ap['best_appt_datetime']
        del ap['instruction']
        del ap['time_point_status_id']
        try:
            Appointment.objects.get(id=ap.get('id'))
        except Appointment.DoesNotExist:
            appointment = Appointment(**ap)
            appointment.save_base(raw=True)
            count += 1
        else:
            already_created += 1
    
print(f'Total created subject appointment {count}')
print(f'Already created {already_created}')
print("Expected: ", len(ap_data))
print("Existing: ", Appointment.objects.all().count())

Total created subject appointment 0
Already created 9627
Expected:  9627
Existing:  9627


In [34]:
# Subject visit
from datetime import timedelta
from td_maternal.models import MaternalVisit
from edc_appointment.models import Appointment
from django.db.utils import IntegrityError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/maternal_visits.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


sv_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    sv_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for sv in sv_data:
    if not sv.get('subject_identifier')[-3:] == '-10':
        del sv['_state']
        del sv['_appointment_cache']
        sv.update(
            created=parser.parse(sv.get('created')).replace(tzinfo=tz),
            id=uuid.UUID(sv.get('id')),
        )
        if sv['report_datetime']:
            sv.update(
                report_datetime=parser.parse(sv.get('report_datetime')).replace(tzinfo=tz))
        else:
            sv.update(report_datetime=None)
        if sv['modified']:
            sv.update(
                modified=parser.parse(sv.get('modified')).replace(tzinfo=tz))
        else:
            sv.update(modified=None)
        
        
        if sv['last_alive_date']:
            sv.update(
                last_alive_date=parser.parse(sv.get('last_alive_date')).replace(tzinfo=tz).date())
        else:
            sv.update(last_alive_date=sv['created'])
        try:
            appointment = Appointment.objects.get(id=sv.get('appointment_id'))
        except Appointment.DoesNotExist:
            print(sv.get('appointment_id'))
            count_missing += 1
        else:
            sv.update(
                appointment=appointment,
                schedule_name=appointment.schedule_name,
                visit_schedule_name=appointment.visit_schedule_name,
                site=appointment.site,
                visit_code=appointment.visit_code
            )
        del sv['export_change_type']
        del sv['export_uuid']
        del sv['exported']
        del sv['exported_datetime']
        try:
            MaternalVisit.objects.get(id=sv.get('id'))
        except MaternalVisit.DoesNotExist:
            maternal_visit = MaternalVisit(**sv)
            maternal_visit.save_base(raw=True)
            count += 1
        else:
            already_created += 1
print(f'Total created subject visits {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sv_data))
print("Existing: ", MaternalVisit.objects.all().count())

Total created subject visits 0
Already created 4001
Expected:  4001
Existing:  4001


In [46]:
# Maternal Labour Delivery
from datetime import timedelta
from td_maternal.models import MaternalLabourDel
from django.db.utils import IntegrityError

fname = '/Users/coulsonkgathi/source/td_data_files/20190618/maternal_labour_del.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


mld_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    mld_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for mld in mld_data:
    del mld['_state']
    mld.update(
        created=parser.parse(mld.get('created')).replace(tzinfo=tz),
        id=uuid.UUID(mld.get('id')),
    )
    if mld['report_datetime']:
        mld.update(
            report_datetime=parser.parse(mld.get('report_datetime')).replace(tzinfo=tz))
    else:
        mld.update(report_datetime=None)
    if mld['arv_initiation_date']:
        mld.update(
            arv_initiation_date=parser.parse(mld.get('arv_initiation_date')).replace(tzinfo=tz).date())
    else:
        mld.update(arv_initiation_date=None)
    if mld['modified']:
        mld.update(
            modified=parser.parse(mld.get('modified')).replace(tzinfo=tz))
    else:
        mld.update(modified=None)
    if mld['delivery_datetime']:
        mld.update(
            delivery_datetime=parser.parse(mld.get('delivery_datetime')).replace(tzinfo=tz))
    else:
        mld.update(delivery_datetime=None)
    del mld['export_change_type']
    del mld['export_uuid']
    del mld['exported']
    del mld['exported_datetime']
    del mld['consent_version']
    del mld['registered_subject_id']
    try:
        MaternalLabourDel.objects.get(id=mld.get('id'))
    except MaternalLabourDel.DoesNotExist:
        maternal_labour_del = MaternalLabourDel(**mld)
        maternal_labour_del.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created maternal labour delivery {count}')
print(f'Already created {already_created}')
print("Expected: ", len(mld_data))
print("Existing: ", MaternalLabourDel.objects.all().count())

Total created maternal labour delivery 0
Already created 462
Expected:  462
Existing:  462
